In [1]:
import sqlite3
import pandas as pd

In [2]:
PATH_DB = './spotify.sqlite'
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [3]:
con = sqlite3.connect(f'file:{PATH_DB}?mode=ro', uri=True)
df_db = pd.read_sql_query('''
    SELECT
        name, sql
    FROM
        sqlite_master
''', con)
con.close()
# View database before optimization
df_db

,name,sql
0,albums,"CREATE TABLE albums ([id], [name], [album_group], [album_type], [release_date], [popularity])"
1,artists,"CREATE TABLE artists ([name], [id], [popularity], [followers])"
2,audio_features,"CREATE TABLE audio_features ([id], [acousticness], [analysis_url], [danceability], [duration], [energy], [instrumentalness], [key], [liveness], [loudness], [mode], [speechiness], [tempo], [time_signature], [valence])"
3,genres,CREATE TABLE genres ([id])
4,r_albums_artists,"CREATE TABLE r_albums_artists ([album_id], [artist_id])"
5,r_albums_tracks,"CREATE TABLE r_albums_tracks ([album_id], [track_id])"
6,r_artist_genre,"CREATE TABLE r_artist_genre ([genre_id], [artist_id])"
7,r_track_artist,"CREATE TABLE r_track_artist ([track_id], [artist_id])"
8,tracks,"CREATE TABLE tracks ([id], [disc_number], [duration], [explicit], [audio_feature_id], [name], [preview_url], [track_number], [popularity], [is_playable])"


In [4]:
con = sqlite3.connect(PATH_DB)
cur = con.cursor()

print('Resetting database to initial state...')
cur.execute('DROP TABLE IF EXISTS audio_features_indexed')
cur.execute('DROP TABLE IF EXISTS tracks_indexed')
cur.execute('DROP TABLE IF EXISTS albums_indexed')
cur.execute('DROP TABLE IF EXISTS artists_indexed')
con.commit()

# artists_indexed
print('\nartists_indexed')
cur.execute('''
    CREATE TABLE artists_indexed (
        id TEXT PRIMARY KEY,
        name TEXT NOT NULL,
        followers INTEGER,
        popularity INTEGER NOT NULL
    )
''')
print('Copying data...')
cur.execute('''
    INSERT INTO artists_indexed (
        id,
        name,
        followers,
        popularity
    )
    SELECT
        id,
        name,
        followers,
        popularity
    FROM
        artists
''')
print('Indexing table...')
cur.execute('CREATE INDEX idx_artists_indexed_name ON artists_indexed(name)')
cur.execute('CREATE INDEX idx_artists_indexed_followers ON artists_indexed(followers)')
cur.execute('CREATE INDEX idx_artists_indexed_popularity ON artists_indexed(popularity)')
con.commit()

# albums_indexed
print('\nalbums_indexed')
cur.execute('''
    CREATE TABLE albums_indexed (
        id TEXT PRIMARY KEY,
        name TEXT NOT NULL,
        album_type TEXT,
        release_date TEXT,
        popularity INTEGER NOT NULL
    )
''')
print('Copying data...')
cur.execute('''
    INSERT INTO albums_indexed (
        id,
        name,
        album_type,
        release_date,
        popularity
    )
    SELECT
        id,
        name,
        album_type,
        release_date,
        popularity
    FROM
        albums
''')
print('Indexing table...')
cur.execute('CREATE INDEX idx_albums_indexed_name ON albums_indexed(name)')
cur.execute('CREATE INDEX idx_albums_indexed_album_type ON albums_indexed(album_type)')
cur.execute('CREATE INDEX idx_albums_indexed_release_date ON albums_indexed(release_date)')
cur.execute('CREATE INDEX idx_albums_indexed_popularity ON albums_indexed(popularity)')
con.commit()

# tracks_indexed
print('\ntracks_indexed')
cur.execute('''
    CREATE TABLE tracks_indexed (
        id TEXT PRIMARY KEY,
        name TEXT NOT NULL,
        duration INTEGER,
        explicit INTEGER,
        audio_feature_id TEXT NOT NULL,
        popularity INTEGER NOT NULL,
        UNIQUE(audio_feature_id)
    )
''')
print('Copying data...')
cur.execute('''
    INSERT INTO tracks_indexed (
        id,
        name,
        duration,
        explicit,
        audio_feature_id,
        popularity
    )
    SELECT
        tracks.id,
        tracks.name,
        tracks.duration,
        tracks.explicit,
        tracks.audio_feature_id,
        tracks.popularity
    FROM
        tracks
    WHERE
        id = audio_feature_id
''')
print('Indexing table...')
cur.execute('CREATE INDEX idx_tracks_indexed_name ON tracks_indexed(name)')
cur.execute('CREATE INDEX idx_tracks_indexed_duration ON tracks_indexed(duration)')
cur.execute('CREATE INDEX idx_tracks_indexed_explicit ON tracks_indexed(explicit)')
cur.execute('CREATE UNIQUE INDEX idx_tracks_indexed_audio_feature_id ON tracks_indexed(audio_feature_id)')
cur.execute('CREATE INDEX idx_tracks_indexed_popularity ON tracks_indexed(popularity)')
con.commit()

# audio_features_indexed
print('\naudio_features_indexed')
cur.execute('''
    CREATE TABLE audio_features_indexed (
        id TEXT PRIMARY KEY,
        analysis_url TEXT,
        loudness REAL,
        tempo REAL,
        time_signature INTEGER,
        key INTEGER,
        mode INTEGER,
        acousticness REAL,
        danceability REAL,
        energy REAL,
        instrumentalness REAL,
        liveness REAL,
        speechiness REAL,
        valence REAL,
        UNIQUE(analysis_url)
    )
''')
print('Copying data...')
cur.execute('''
    INSERT INTO audio_features_indexed (
        id,
        analysis_url,
        loudness,
        tempo,
        time_signature,
        key,
        mode,
        acousticness,
        danceability,
        energy,
        instrumentalness,
        liveness,
        speechiness,
        valence
    )
    SELECT
        id,
        analysis_url,
        loudness,
        tempo,
        time_signature,
        key,
        mode,
        acousticness,
        danceability,
        energy,
        instrumentalness,
        liveness,
        speechiness,
        valence
    FROM
        audio_features
''')
print('Indexing table...')
cur.execute('CREATE UNIQUE INDEX indexed_analysis_url ON audio_features_indexed(analysis_url)')
cur.execute('CREATE INDEX indexed_loudness ON audio_features_indexed(loudness)')
cur.execute('CREATE INDEX indexed_tempo ON audio_features_indexed(tempo)')
cur.execute('CREATE INDEX indexed_time_signature ON audio_features_indexed(time_signature)')
cur.execute('CREATE INDEX indexed_key ON audio_features_indexed(key)')
cur.execute('CREATE INDEX indexed_mode ON audio_features_indexed(mode)')
cur.execute('CREATE INDEX indexed_acousticness ON audio_features_indexed(acousticness)')
cur.execute('CREATE INDEX indexed_danceability ON audio_features_indexed(danceability)')
cur.execute('CREATE INDEX indexed_energy ON audio_features_indexed(energy)')
cur.execute('CREATE INDEX indexed_instrumentalness ON audio_features_indexed(instrumentalness)')
cur.execute('CREATE INDEX indexed_liveness ON audio_features_indexed(liveness)')
cur.execute('CREATE INDEX indexed_speechiness ON audio_features_indexed(speechiness)')
cur.execute('CREATE INDEX indexed_valence ON audio_features_indexed(valence)')
con.commit()

# Clean up
print('\nCleaning up database...')
cur.execute('VACUUM')
con.commit()

cur.close()
con.close()

Resetting database to initial state...

artists_indexed
Copying data...
Indexing table...

albums_indexed
Copying data...
Indexing table...

tracks_indexed
Copying data...
Indexing table...

audio_features_indexed
Copying data...
Indexing table...

Cleaning up database...


OperationalError: database or disk is full

In [5]:
con = sqlite3.connect(f'file:{PATH_DB}?mode=ro', uri=True)
df_db = pd.read_sql_query('''
    SELECT
        name, sql
    FROM
        sqlite_master
''', con)
con.close()
# View database after optimization
df_db

,name,sql
0,albums,"CREATE TABLE albums ([id], [name], [album_group], [album_type], [release_date], [popularity])"
1,artists,"CREATE TABLE artists ([name], [id], [popularity], [followers])"
2,audio_features,"CREATE TABLE audio_features ([id], [acousticness], [analysis_url], [danceability], [duration], [energy], [instrumentalness], [key], [liveness], [loudness], [mode], [speechiness], [tempo], [time_signature], [valence])"
3,genres,CREATE TABLE genres ([id])
4,r_albums_artists,"CREATE TABLE r_albums_artists ([album_id], [artist_id])"
5,r_albums_tracks,"CREATE TABLE r_albums_tracks ([album_id], [track_id])"
6,r_artist_genre,"CREATE TABLE r_artist_genre ([genre_id], [artist_id])"
7,r_track_artist,"CREATE TABLE r_track_artist ([track_id], [artist_id])"
8,tracks,"CREATE TABLE tracks ([id], [disc_number], [duration], [explicit], [audio_feature_id], [name], [preview_url], [track_number], [popularity], [is_playable])"
9,artists_indexed,"CREATE TABLE artists_indexed (\n id TEXT PRIMARY KEY,\n name TEXT NOT NULL,\n followers INTEGER,\n popularity INTEGER NOT NULL\n )"
